In [1]:
import numpy as np
import pandas as pd
import docx2txt
import re
import json
from os import listdir
from collections import Counter
import copy

In [2]:
# read in word file
play_text = docx2txt.process('Word_Docs/133.docx')

In [3]:
def parse_characters(play_text):
    play_text = play_text.replace('\n\n', '\n')
    characters_summary = {}
    characters = play_text[play_text.find('LES ACTEURS')+len('LES ACTEURS'):
                                   play_text.find('ACTE 1')]
    noise = ['', ' ', '\xa0', '-', '–', '/']
    dramatic_characters = [character.strip() for character in characters.split('\n') if character not in noise]
    for character in dramatic_characters:
        collective_number = re.findall('\d', character)
        if len(collective_number) == 0:
            characters_summary[character] = {'collective_number': None}
        else:
            number = collective_number[0]
            character = character.replace(number, '').strip()
            characters_summary[character] = {'collective_number': int(number)}
    
    return characters_summary

In [4]:
def speach_analysis(scene_characters, characters_dict):
    speach_dict = {'number_speaking_characters': 0, 'number_non_speaking_characters':0}
    characters = [key for key in scene_characters.keys() if key not in ['num_speakers']]
    for character in  characters:
        collective_num = characters_dict[character]['collective_number']
        if scene_characters[character] == 'speaking' and collective_num == None:
            speach_dict['number_speaking_characters'] += 1
        elif scene_characters[character] == 'speaking' and collective_num != None:
            speach_dict['number_speaking_characters'] += characters_dict[character]['collective_number']
        elif scene_characters[character] == 'non_speaking':
            speach_dict['number_non_speaking_characters'] += 1
    speach_dict['percentage_non_speaking'] = round((speach_dict['number_non_speaking_characters'] / 
                                                    len(characters)) * 100, 3)              
    return speach_dict

In [5]:
def character_parsing(names, characters):
    scene_characters = {}
    for name in names:                 
        symbols = ['-', '–', '*', '/']
        for symbol in symbols:
            name = name.replace(symbol, '')
        if name.lower().count('молчит')> 0 :
            name = name.lower().replace('молчит', '').upper()
            speaking_status = 'non_speaking'
        else:
            speaking_status = 'speaking'
        name = name.strip()
        if name.isdigit() == False and name != '':
            scene_characters[name] = speaking_status
   
    scene_characters['num_speakers'] = speach_analysis(scene_characters, characters)['number_speaking_characters']
    scene_characters['perc_non_speakers'] = speach_analysis(scene_characters, characters)['percentage_non_speaking']
    
    return scene_characters

In [6]:
def parse_scenes(scenes,  characters):
    scene_summary = {}
    noise = ['', ' ', '\xa0', '-', '–', '/']
    extra_scene_number = 1
    regular_num = 1
    for scene in scenes:
        names = [name.strip() for name in scene[1:].split('\n') if name not in noise]
        if scene[0].isdigit() and scene[1:5].count('*') == 0:
            scene_name = str(regular_num) +'_regular'
            regular_num += 1
            extra_scene_number = 1
        elif scene[0].isdigit() and scene[1:5].count('*') > 0:
            scene_name =  str(regular_num) + '_no_change'
            regular_num += 1
            extra_scene_number = 1
        else: 
            scene_name =  str(regular_num - 1)+'.' + str(extra_scene_number) +'_extra'
            extra_scene_number += 1
        scene_summary[scene_name] = character_parsing(names,  characters)
    
    return scene_summary

In [7]:
def process_play_summary(play_data, play_text):
    play_data['characters'] = parse_characters(play_text)
    play_summary = {}
    noise = ['', ' ', '\xa0', '-', '–', '/']
    acts = [act for act in play_text[play_text.find('ACTE 1'):].split('ACTE') if act not in noise]
    for act_num, act in enumerate(acts, 1):
        scenes =   [scene.strip() for scene in act.split('SCENE')][1:]
        play_summary['act_'+ str(act_num)] = parse_scenes(scenes,  play_data['characters'])
    play_data['play_summary'] = play_summary
    
    return play_data

In [8]:
def estimate_number_scenes(scene_summary):
    """
    The function calcualtes the number of scenes per text and per Iarkho (i.e., as marked by actual dramatic character
    entrances and exits).
    Params:
        scene_summary - a dictionary output of the parse_play function.
    Returns:
        total_number_scenes_per_text - number of scenes as they are printed
        total_number_scenes_iarkho - number of scnes per Iarkho, which he calls mobility coefficient (MC)
    """
    total_number_scenes_per_text = 0
    total_number_scenes_iarkho = 0
    for key in scene_summary.keys():
        # get the number of scenes as it is printed in the text
        total_number_scenes_per_text+=len([scene for scene in scene_summary[key].keys() if scene.count('extra')==0])
        # count scenes as marked by actual entrances and exits                                  
        total_number_scenes_iarkho+=len([scene for scene in scene_summary[key].keys() if scene.count('no_change')==0])
    
    return total_number_scenes_per_text, total_number_scenes_iarkho

In [9]:
def number_speaking_no_change_case(previous_scene, no_change_scene):
    speaking_set = set()
    non_speaking_set = set()
    for key in previous_scene.keys():
        if previous_scene[key] =='speaking' or no_change_scene[key]=='speaking':
            speaking_set.add(key)
        if previous_scene[key] =='non_speaking' or no_change_scene[key]=='non_speaking':
            non_speaking_set.add(key)
    num_non_speaking = len(non_speaking_set.difference(speaking_set))
    num_speaking = len(speaking_set)
    perc_non_speaking = round((num_non_speaking / (num_non_speaking + num_speaking)) * 100, 3)
    
    return num_speaking, perc_non_speaking

In [10]:
def combine_no_change_scenes(play_summary):
    which_to_exclude = []
    speakers = []
    perc_non_speakers = []
    for act in play_summary.keys():
        analysed_scenes = []
        for scene in list(play_summary[act].keys()):
            if scene.count('no_change') > 0:

                num_speaking, perc_non_speaking = number_speaking_no_change_case(
                                                  play_summary[act][analysed_scenes[-1]],
                                                  play_summary[act][scene])
                speakers.append(num_speaking)
                perc_non_speakers.append(perc_non_speaking)
                which_to_exclude.append((act, scene, analysed_scenes[-1]))
            analysed_scenes.append(scene)
    
    return which_to_exclude, speakers, perc_non_speakers

In [11]:
def remove_combined_scenes(play_dict, values_to_exclude):
    """
    The function removes info about scenes that we have previously combined and calculated combined data
    for in cases when there was no change in character cast.
    Params:
        play_dict - a dictionary with speakers for each scene.
        values_to_exlude - a list of typles where the first value is the act and the other values are scenes.
        
    Returns:
        play_dict - without exluded scenes.
    """
    for value in values_to_exclude:
        result = {key : val for key, val in play_dict[value[0]].items() 
                        if key not in value[1:]}
        play_dict[value[0]] = result
        
    return play_dict

In [12]:
def preprocess_play_summary(play_summary_copy):
    values_to_exclude, speakers, perc_non_speakers = combine_no_change_scenes(play_summary_copy)
    play_summary_updated = remove_combined_scenes(play_summary_copy, values_to_exclude)
    for key in play_summary_updated.keys():
        for scene in play_summary_updated[key]:
            speakers.append((play_summary_updated[key][scene]['num_speakers']))
            perc_non_speakers.append(round(play_summary_updated[key][scene]['perc_non_speakers'], 3))
    
    return speakers, perc_non_speakers

In [13]:
def speech_distribution_iarkho(play_summary_copy):
    """
    The function creates speech distrubution per Iarkho, i.e., the number of speaking characters by number of scenes.
    Params:
        play_summary - a dictionary output by parse_play function.
    Returns:
        speech_distribution - a list of tuples were the 0 element is the number of speaking characters
                              and the 1 element is the number of scenes with such number of speaking characters.
    """
    
    speakers, perc_non_speakers = preprocess_play_summary(play_summary_copy)
    counter = Counter
    counted = counter(speakers)
    speech_distribution = sorted(counted.items(), key=lambda pair: pair[0], reverse=False)
    speech_types = percentage_of_each_speech_type(speech_distribution)
    av_perc_non_speakers = round(np.mean((perc_non_speakers)), 3)
    
    return speech_distribution, speech_types, av_perc_non_speakers

In [14]:
def sigma_iarkho(variants, weights):  
    """ 
    The function allows calculating standard range following iarkho's procedure. 
    Parameters: 
        variants - a list with distinct variants in the ascending order, e.g. [1, 2, 3, 4, 5] 
        weights - a list of weights corresponding to these variants, e.g. [20, 32, 18, 9, 1] 
    Returns: 
        sigma - standard range per iarkho 
    """  
    weighted_mean_variants = np.average(variants, weights=weights)  
    differences_squared = [(variant - weighted_mean_variants)**2 for variant in variants] 
    weighted_mean_difference = np.average(differences_squared, weights=weights)  
    sigma = weighted_mean_difference**0.5  
      
    return sigma 

In [15]:
def percentage_of_each_speech_type(speech_distribution):
    """
    The function calculates the percentage of each speech type (monologue, duologue, non-duologue (meaning not two
    speakers), and over-two speakers) of the total accross all speech types.
    Params:
        speech_distibution - number of scenes with a specified number of speakers.
    Returns:
        speech_types - a dictionary with percentages corresponding to each speech type.
    """
    speech_types = {}
    total_scenes = np.sum([speech_type[1] for speech_type in  speech_distribution])
    speech_types['perc_monologue'] = np.round((np.sum([speech_type[1] for speech_type in  speech_distribution 
                                    if speech_type[0] ==1]) / total_scenes) *100, 2)
    speech_types['perc_duologue'] = np.round((np.sum([speech_type[1] for speech_type in  speech_distribution 
                                    if speech_type[0] == 2])/ total_scenes) * 100, 2)
    speech_types['perc_non_duologue'] = np.round((np.sum([speech_type[1] for speech_type in  speech_distribution 
                                        if speech_type[0] != 2])/ total_scenes) * 100, 2)
    speech_types['perc_over_two_speakers'] = np.round((np.sum([speech_type[1] for speech_type in  speech_distribution 
                                             if speech_type[0] > 2])/ total_scenes) * 100, 2)
    
    return speech_types

In [16]:
def number_present_characters(play_dictionary):
    """
    The function calculates the number of characters present in the play. If a character is listed in cast, but doesn't
    appear on stage, he/she doesn't count.
    Params:
        play_dictionary - a dictioanry with data for the play, which includes the characters present in each scene.
    Returns:
        total_number_present_characters - int.
    """
    all_present_characters = set()
    for key in play_dictionary['play_summary'].keys():
        for scene in play_dictionary['play_summary'][key]:
            for item in play_dictionary['play_summary'][key][scene].keys():
                if item != 'num_speakers' and item != 'perc_non_speakers':
                    all_present_characters.add(item)
    total_number_present_characters = 0
    if len(set(all_present_characters).difference(set(play_dictionary['characters']))) > 0:
        print('Error. Incorrect character name present in a scene.')
    appearing_on_stage = set(play_dictionary['characters']).intersection(all_present_characters)
    for character in appearing_on_stage: 
        coll_number = play_dictionary['characters'][character]['collective_number']
        # if there is a collective number for this character
        if coll_number:
            total_number_present_characters += int(coll_number)
        else:
            total_number_present_characters += 1

    return total_number_present_characters

In [17]:
def process_speakers_features(play_data, metadata_dict):
    """
    Iarkho's features described in Iarkho's work on the evolution of 5-act tragedy in verse.
    """
    metadata_dict['num_present_characters'] = number_present_characters(play_data)
    metadata_dict['num_scenes_text'] = estimate_number_scenes(play_data['play_summary'])[0]
    metadata_dict['num_scenes_iarkho'] = estimate_number_scenes(play_data['play_summary'])[1]
    play_summary_copy = copy.deepcopy(play_data['play_summary'])
    distribution, speech_types, non_speakers = speech_distribution_iarkho(play_summary_copy)
    metadata_dict['speech_distribution'] = distribution
    metadata_dict['percentage_monologues'] = speech_types['perc_monologue']
    metadata_dict['percentage_duologues'] = speech_types['perc_duologue']
    metadata_dict['percentage_non_duologues'] = speech_types['perc_non_duologue']
    metadata_dict['percentage_above_two_speakers'] = speech_types['perc_over_two_speakers']
    metadata_dict['av_percentage_non_speakers'] = non_speakers
    metadata_dict['sigma_iarkho'] = round(sigma_iarkho(
                                    [item[0] for item in metadata_dict['speech_distribution']],
                                    [item[1] for item in metadata_dict['speech_distribution']]), 3)
    
    return metadata_dict

In [18]:
def percentage_of_scenes_discont_change(play_data, metadata_dict):
    number_scenes = metadata_dict['num_scenes_iarkho']
    characters = []
    num_scenes_with_disc_character_change = 0
    for act in play_data['play_summary'].keys():
        for entry in play_data['play_summary'][act].values():
            new_cast = [item for item in entry.keys() if 
                               item not in ['num_speakers', 'perc_non_speakers', 'num_utterances']]
            if len(characters) > 0:
                if len(set(new_cast).intersection(set(characters[-1]))) == 0:
                    num_scenes_with_disc_character_change += 1
            characters.append(new_cast)
    perc_disc = round((num_scenes_with_disc_character_change /number_scenes) * 100, 3) 
    metadata_dict['number_scenes_with_discontinuous_change_characters'] = num_scenes_with_disc_character_change
    metadata_dict['percentage_scenes_with_discontinuous_change_characters'] = perc_disc
    
    return metadata_dict

In [19]:
def metadata_processing(play_string, play_data):
    """
    Process all play features in stages
    """
    metadata_dict = {}
    for process in [process_speakers_features, percentage_of_scenes_discont_change]:
        metadata_dict = process(play_data, metadata_dict)

    return metadata_dict

In [20]:
def add_play_info(metadata):
    """
    Update play metadata from the metadata_df.
    """
    play_data = {}
    play_data['title'] = metadata[0][0]
    first_name = metadata[0][2]
    if type(first_name) != float:
        play_data['author'] = str(metadata[0][1] + ', ' + metadata[0][2]).replace('\xa0', '')
    else:
        play_data['author'] = metadata[0][1].replace('\xa0', '') 
    play_data['date'] = metadata[0][3]
    
    return play_data

In [21]:
def process_play(file_name, metadata_df,  input_path):
    """
    The function parses a txt file and creates a summary with features and metadata for the play.
    Params:
        file_name - a string, name of the file with the play text.
        metadata_df - a dataframe containing the info about the play.
    Returns:
        play_data - a dictionary with detailed play summary by scenes, metadata, and features
    """
    play_index = file_name.replace(input_path, '').replace('.docx', '').replace('F_', '')
    play_meta = metadata_df[metadata_df['index']=='F_' + play_index][['title', 'last_name', 
                                                            'first_name', 'date']].values                                                      
    comedy = docx2txt.process(file_name)
    number_acts = int(metadata_df[metadata_df['index']=='F_'+play_index]['num_acts'].values[0])
    play_data = add_play_info(play_meta)
    play_data = process_play_summary(play_data, comedy)
    play_data['metadata'] = metadata_processing(play_text, play_data)
    
    return play_data

In [22]:
def process_all_plays(input_directory, output_path, metadata_path):
    """
    The function allows to process all files in a specified directory.
    Params:
        input_directory - the path to the folder containing the txt files
        output_path - directory in which the json summaries will be saved.
        metadata_path - path to the metadata file, a tab-delimited txt file with informtion about all plays.
        regex_pattern - a regex pattern which identifies dramatic character names.
    Returns:
        no returns, the files will be saved in output_path directory.
    """
    all_files = [f for f in listdir(input_directory) if f.count('.docx')>0]
    metadata_df = pd.read_csv(metadata_path, sep='\t')
    for file in all_files:
        print(file)
        play_data_dict = process_play(input_directory+file, metadata_df, input_directory)
        json_name = output_path + 'F_' + str(file.replace('.docx', '.json')) 
        with open(json_name, 'w') as fp:
            json.dump(play_data_dict, fp, ensure_ascii=False, indent=2)

In [23]:
process_all_plays('Word_Docs/', 'Play_Jsons/', 'French_Comedies.tsv')

221.docx
108.docx
260.docx
85.docx
165.docx
173.docx
124.docx
295.docx
104.docx
145.docx
283.docx
66.docx
217.docx
186.docx
31.docx
240.docx
305.docx
190.docx
256.docx
201.docx
71.docx
129.docx
257.docx
191.docx
304.docx
282.docx
144.docx
105.docx
294.docx
92.docx
125.docx
172.docx
164.docx
133.docx
261.docx
236.docx
51.docx
109.docx
220.docx
298.docx
277.docx
293.docx
102.docx
114.docx
285.docx
211.docx
180.docx
246.docx
303.docx
196.docx
179.docx
250.docx
207.docx
159.docx
227.docx
118.docx
289.docx
266.docx
83.docx
134.docx
163.docx
175.docx
122.docx
174.docx
162.docx
288.docx
267.docx
230.docx
226.docx
158.docx
271.docx
206.docx
98.docx
77.docx
251.docx
197.docx
178.docx
247.docx
302.docx
181.docx
61.docx
139.docx
284.docx
115.docx
292.docx
154.docx
205.docx
74.docx
252.docx
194.docx
301.docx
244.docx
35.docx
182.docx
213.docx
62.docx
9.docx
268.docx
141.docx
229.docx
100.docx
291.docx
157.docx
97.docx
209.docx
198.docx
161.docx
248.docx
81.docx
264.docx
233.docx
225.docx
224.docx


process_play('Word_Docs/104.docx', pd.read_csv('French_Comedies.tsv', sep='\t'), 'Word_Docs/')

character_sets = []
for act in play_data['play_summary'].keys():
    for scene in play_data['play_summary'][act].keys():
        new_character_set = set([key for key in play_data['play_summary'][act][scene].keys() 
               if key not in ['num_speakers', 'perc_non_speakers']])
        character_sets.append(new_character_set)
        if len(character_sets) > 1 and new_character_set == character_sets[-2] and scene.count('no_change')==0 and scene.count('1')==0:
            print(act, scene, new_character_set)